In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
seed = 2
from xgboost import XGBClassifier
import pickle

In [ ]:
train = pd.read_csv('./data/new_train.csv')

In [ ]:
sample_size = int(1e4)
sample = train[train.target == 1].sample(n=sample_size/2, random_state=seed)
sample = sample.append( train[train.target == 0].sample(n=sample_size/2, random_state=seed))
sample = sample.sample(frac=1, random_state=seed).reset_index(drop=True)
#sample.to_csv('./data/train_sample.csv', index=False)

In [ ]:
target = train.target
train.drop(['target','id'], inplace=True, axis=1)
train.drop([ col for col in train.columns if col.startswith('ps_cont') ],axis=1, inplace=True)

In [ ]:
with open('./data/OneHotEncoder.clf', 'rb') as f:
    encoders = pickle.load(f)

In [ ]:
enc_train = None

for feature,encoder in zip(train.columns,encoders) :
    encoded = encoder.transform(train[feature].values.reshape(-1,1))
    if enc_train is None :
        enc_train = encoded
    else :
        enc_train = np.concatenate((enc_train, encoded), axis=1)

In [ ]:
del train

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(enc_train, target, test_size=.1, random_state=seed)

In [ ]:
model = XGBClassifier(
	objective = 'binary:logistic',
	n_jobs = 4,
	random_state = seed,
	eval_metric = "error",
	#-------
	max_depth = 1,
	min_child_weight = 7,
	#----------
	gamma = 0,
	#----------
	subsample = 0.4,
	colsample_bytree = 0.4,
	#------------
	scale_pos_weight = 3,
	#-----------
	reg_alpha = 0,
	reg_lambda = 1,
	#------------
	learning_rate = 0.1

	#n_estimators = 1000,
	)

In [ ]:
with open('./data/parameters.pkl','rb') as f:
    p = pickle.load(f)

In [ ]:
model = XGBClassifier(**p)

In [ ]:
model.fit(enc_train, target)

In [ ]:
with open('./data/model.pkl', 'wb') as f:
    pickle.dump(file=f, obj=model)

In [ ]:
with open('./data/model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
pred = model.predict(enc_train)

In [ ]:
from sklearn.metrics import classification_report
print classification_report(y_true=target, y_pred=pred)

In [ ]:
train.ps_cat_ind_02.value_counts()

In [ ]:
del x_train, y_train